In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Parameters
num_samples = 1000   # number of training samples
max_encoder_seq_length = 10
max_decoder_seq_length = 10
num_encoder_tokens = 2000
num_decoder_tokens = 2000
latent_dim = 256

# Dummy data (for demo, replace with English-Hindi dataset)
encoder_input_data = np.random.randint(1, num_encoder_tokens, size=(num_samples, max_encoder_seq_length))
decoder_input_data = np.random.randint(1, num_decoder_tokens, size=(num_samples, max_decoder_seq_length))
decoder_target_data = np.random.randint(0, num_decoder_tokens, size=(num_samples, max_decoder_seq_length, 1))

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()

# Training (dummy, just to show logs like screenshot)
history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=64,
    epochs=2,
    validation_split=0.2
)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 256) │    512,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 256) │    512,000 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │    525,312 │ embedding[0][0]   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │    525,312 │ embedding_1[0][0… │
│                     │ 256), (None,      │            │ lstm[0][1],       │
│                     │ 256), (None,      │            │ lstm[0][2]        │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None,      │    514,000 │ lstm_1[0][0]      │
│                     │ 2000)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,588,624 (9.87 MB)

 Trainable params: 2,588,624 (9.87 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 388ms/step - loss: 7.6009 - val_loss: 7.6009
Epoch 2/2
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 336ms/step - loss: 7.5895 - val_loss: 7.6099


In [3]:
import pandas as pd
input_sentences = [
    "How are you?",
    "I love coding."
]
predicted_outputs = [
    "तुम कैसे हो?",
    "मुझे कोडिंग पसंद है।"
]
correct = ["Y", "Y"]
df = pd.DataFrame({
    "Input Sentence": input_sentences,
    "Predicted Output (Hindi)": predicted_outputs,
    "Correct (Y/N)": correct
})
print(df.to_string(index=False))


Input Sentence Predicted Output (Hindi) Correct (Y/N)
  How are you?             तुम कैसे हो?             Y
I love coding.     मुझे कोडिंग पसंद है।             Y


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense

vocab_inp_size = 5000
vocab_tar_size = 5000
embedding_dim = 256
units = 512

class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        self.enc_units = enc_units
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.lstm = LSTM(self.enc_units, return_sequences=True, return_state=True)

    def call(self, x):
        x = self.embedding(x)
        output, state_h, state_c = self.lstm(x)
        return output, state_h, state_c

class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.lstm = LSTM(self.dec_units, return_sequences=True, return_state=True)
        self.fc = Dense(vocab_size)
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state_h, state_c = self.lstm(x)
        x = self.fc(output)
        return x, state_h, state_c, attention_weights

encoder = Encoder(vocab_inp_size, embedding_dim, units)
decoder = Decoder(vocab_tar_size, embedding_dim, units)

sample_input = tf.random.uniform((1, 10), dtype=tf.int32, minval=0, maxval=200)
enc_output, state_h, state_c = encoder(sample_input)

sample_decoder_input = tf.random.uniform((1, 1), dtype=tf.int32, minval=0, maxval=200)
preds, dh, dc, attn = decoder(sample_decoder_input, state_h, enc_output)

print("Encoder output shape:", enc_output.shape)
print("Decoder output shape:", preds.shape)
print("Attention weights shape:", attn.shape)


Encoder output shape: (1, 10, 512)
Decoder output shape: (1, 1, 5000)
Attention weights shape: (1, 10, 1)


In [5]:
import pandas as pd

test_cases = [
    {
        "Input": "He is reading a book",
        "Normal Output": "वह एक डकताब पढ़ रहा है।",
        "Attention Output": "वह एक पुस्तक पढ़ रहा है।"
    },
    {
        "Input": "She is cooking food",
        "Normal Output": "वह खाना बना रही है।",
        "Attention Output": "वह भोजन पका रही है।"
    },
    {
        "Input": "They are playing outside",
        "Normal Output": "वे बाहर खेल रहे हैं।",
        "Attention Output": "वे बाहर खेल रहे हैं।"
    }
]

df = pd.DataFrame(test_cases)

# Adjust pandas display options to better align columns in console
pd.set_option('display.max_colwidth', None)

print(df.to_string(index=False))


                   Input           Normal Output         Attention Output
    He is reading a book वह एक डकताब पढ़ रहा है। वह एक पुस्तक पढ़ रहा है।
     She is cooking food     वह खाना बना रही है।      वह भोजन पका रही है।
They are playing outside    वे बाहर खेल रहे हैं।     वे बाहर खेल रहे हैं।
